A linear demand curve model is defined as follows with $Q$ (demand/quantity) predicted from $P$ (price). Using algebriac manipulation, we can formulate $P$ in terms of $Q$. The total revenue $R$ and marginal revenue $R'$ may also be defined quite easily using substitution and taking the derivative, respectively. Notice the progression from $Q$ to $P$ to $R$ and then finally $R'$.

- $Q = \beta_0 + \beta_1 P$
- $P = \dfrac{1}{\beta_1}Q - \dfrac{\beta_0}{\beta_1}$
- $R = PQ = \dfrac{1}{\beta_1}Q^2 - \dfrac{\beta_0}{\beta_1}Q$
- $R' = \dfrac{\mathrm{d} R}{\mathrm{d} Q} = \dfrac{2}{\beta_1}Q - \dfrac{\beta_0}{\beta_1}$

Revenue optimality is found by setting the marginal cost $C'$ equal to the marginal revenue $R'$ and then solving for $Q$. The $Q$ value we get when $C'$ is known is the optimal quantity. We can then subsitute $Q$ back into the equations above to get the optimal $P$, $R$ and $R'$.

- $C' = \dfrac{2}{\beta_1}Q - \dfrac{\beta_0}{\beta_1}$
- $\beta_1 C'  = 2Q - \beta_0$
- $\beta_0 + \beta_1 C'  = 2Q$
- $\dfrac{\beta_0 + \beta_1 C'}{2}  = Q$

Profit optimality.

- $T = Q(P - C')$
- $T = (\beta_0 + \beta_1 P)(P - C')$, substituting $Q$
- $T = \beta_0 P - \beta_0 C' + \beta_1 P^2 - \beta_1 C'$, expanding using FOIL method
- $T = \beta_1 P^2 + \beta_0 P - (\beta_0 + \beta_1) C'$, grouping polynomial terms
- $T' = 2 \beta_1 P + \beta_0$, taking derivative with respect to $P$
- $0 = 2 \beta_1 P + \beta_0$, optimal point is $T' = 0$
- $-\dfrac{\beta_0}{2 \beta_1} = P$, solving for $P$

In [1]:
import pandas as pd
import numpy as np

np.random.seed(37)

N = 100
P = np.random.normal(40, 20, N) # np.arange(0, 81, 1)
Q = np.random.normal(400 + (-5 * P), 1, P.shape[0]) + np.random.normal(10, 20, N)

df = pd.DataFrame({'p': P, 'q': Q}) \
    .sort_values(['p']) \
    .query('p >= 0') \
    .reset_index(drop=True)

df.shape

(97, 2)

In [2]:
X = df[['p']]
y = df['q']

mc = 15.0

In [3]:
from sklearn.linear_model import LinearRegression

def get_model(X, y):
    m = LinearRegression()
    m.fit(X, y)
    
    return m

def debug_params(model, mc):
    b_0, b_1 = model.intercept_, model.coef_[0]
    qp_params = f'{b_0:.5f} + {b_1:.5f}p'
    
    z_0, z_1 = (-b_0 / b_1), (1 / b_1)
    pq_params = f'{z_0:.5f} + {z_1:.5f}q'
    tr_params = f'{z_0:.5f}q + {z_1:.5f}q^2'
    
    z_0, z_1 = (-b_0 / b_1), 2 * (1 / b_1)
    mr_params = f'{z_0:.5f} + {z_1:.5f}q'
    
    z_0, z_1 = (-b_0 / b_1), 2 * (1 / b_1)
    qo_params = f'({mc:.5f} - {z_0:.5f}) / {z_1:.5f}'
    
    t_params = f'{-b_0 / (2 * b_1):.5f}p'
    
    return {
        'q': qp_params,
        'p': pq_params,
        'tr': tr_params,
        'mr': mr_params,
        'qo': qo_params,
        't': t_params
    }
    
def get_params(model):
    return pd.Series([model.intercept_, model.coef_[0]], ['b_0', 'b_1'])

def get_pq(b_0, b_1):
    return lambda q: (-b_0 / b_1) + (1 / b_1) * q

def get_qp(b_0, b_1):
    return lambda p: b_0 + (b_1 * p)

def get_mr(b_0, b_1):
    return lambda q: (-b_0 / b_1) + (2 * (1 / b_1) * q)

def get_qo(b_0, b_1):
    z_0 = (-b_0 / b_1)
    z_1 = 2 * (1 / b_1)
    return lambda mc: (mc - z_0) / z_1

def get_to(b_0, b_1):
    return -b_0 / (2 * b_1)

def get_funcs(b_0, b_1):
    pq = get_pq(b_0, b_1)
    qp = get_qp(b_0, b_1)
    mr = get_mr(b_0, b_1)
    qo = get_qo(b_0, b_1)
    r = lambda p, q: p * q
    t = lambda p, q, mc: (p * q) - (mc * q)
    
    return pq, qp, mr, qo, r, t

def get_opt(f, mc):
    pq_f, qp_f, mr_f, qo_f, r_f, t_f = f
    
    q_opt = qo_f(mc)
    p_opt = pq_f(q_opt)
    mr_opt = mr_f(q_opt)
    r_opt = r_f(p_opt, q_opt)
    t_opt = t_f(p_opt, q_opt, mc)

    return pd.Series({
        'mc': mc,
        'p': p_opt,
        'q': q_opt,
        'mr': mr_opt,
        'tr': r_opt,
        'pr': t_opt
    })

def optimize(category):
    X, y = get_Xy(category)
    m = get_model(X, y)
    p = get_params(m)
    f = get_funcs(p.b_0, p.b_1)
    mc = get_mc(category)
    
    return {**{'category': category}, **get_opt(f, mc).to_dict()}

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [4]:
m = get_model(X, y)
debug_params(m, mc)

{'q': '402.06115 + -4.83566p',
 'p': '83.14509 + -0.20680q',
 'tr': '83.14509q + -0.20680q^2',
 'mr': '83.14509 + -0.41359q',
 'qo': '(15.00000 - 83.14509) / -0.41359',
 't': '41.57255p'}

In [5]:
f = get_funcs(m.intercept_, m.coef_[0])
get_opt(f, mc)

mc      15.000000
p       49.072545
q      164.763146
mr      15.000000
tr    8085.346941
pr    5613.899751
dtype: float64

In [6]:
get_to(m.intercept_, m.coef_[0])

41.57254526955509

In [7]:
m.predict([[41.57254526955506]])

C:\Users\super\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\super\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([201.03057438])

In [8]:
m.predict([[41.57254526955506]])[0] * (get_to(m.intercept_, m.coef_[0]) - mc)

C:\Users\super\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\super\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


5341.894038341078